In [1]:
! pip install numpy git+https://github.com/makcedward/nlpaug.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/makcedward/nlpaug.git to /tmp/pip-req-build-_q3abzwx
  Running command git clone --filter=blob:none --quiet https://github.com/makcedward/nlpaug.git /tmp/pip-req-build-_q3abzwx
  Resolved https://github.com/makcedward/nlpaug.git to commit 23800cbb9632c7fc8c4a88d46f9c4ecf68a96299
  Preparing metadata (setup.py) ... done
  Created wheel for nlpaug: filename=nlpaug-1.1.11-py3-none-any.whl size=405902 sha256=a89015eb7bd085e4052e3b6e521bf1e3d7038da43ef4e83c3b32546cff624136
  Stored in directory: /tmp/pip-ephem-wheel-cache-wvrgi12z/wheels/90/bc/37/e55b295d36cbaaaf8394dbd355d28e033e236d2bcc7cf77f3a
Successfully built nlpaug


In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.9 MB/s eta 0:00:00


In [9]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00


In [6]:
! pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=20a1349734d23390026d4b4e2029613bcbaf7e40ed4bca136781930926c94865
  Stored in directory: /root/.cache/pip/wheels/ae/e1/6c/5137bc3f35aa130deea71575e165cc4f4f0680a88f3d90a636
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


Algoritmos Caracteres

In [1]:
text = "Eu adoro NLP e treinar modelos como BERT!"

In [2]:

import nlpaug.augmenter.word as naw
import nlpaug.flow as naf

from nlpaug.util import Method

In [4]:
from nlpaug.augmenter.word import WordAugmenter

class M2M100BackTranslationAug(WordAugmenter):
    

    def __init__(self, from_lang='en', to_lang='pt',n_data = 1, num_beams = 10,
        name='M2M100BackTranslationAug'):
        super().__init__(
            action='substitute')
        
        from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

        model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to('cuda')
        tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

        self.src = from_lang
        self.dest = to_lang
        self.model = model
        self.tokenizer = tokenizer
        self.n_data = n_data
        self.num_beams = num_beams

    def substitute(self, data, n=1):

        self.tokenizer.src_lang = self.src
        encoded_hi = self.tokenizer(data, return_tensors="pt", truncation=True, padding=True).to('cuda')
        generated_tokens = self.model.generate(**encoded_hi, forced_bos_token_id=self.tokenizer.get_lang_id(self.dest),
                                               num_return_sequences = self.n_data, num_beams = self.num_beams)
        resultado = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        self.tokenizer.src_lang = self.dest
        encoded_hi2 = self.tokenizer(resultado, return_tensors="pt", truncation=True, padding=True).to('cuda')
        generated_tokens = self.model.generate(**encoded_hi2, forced_bos_token_id=self.tokenizer.get_lang_id(self.src))
        texto = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        return texto

In [7]:
import nlpaug.augmenter.word as naw

augmenter = M2M100BackTranslationAug(from_lang='pt', to_lang='de', n_data=5, num_beams=10)

In [8]:
augmented_texts = augmenter.augment(text, n=3)
print("Original:")
print(text)
print("Augmented Texts:")
print(augmented_texts)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original:
Eu adoro NLP e treinar modelos como BERT!
Augmented Texts:
[['Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de NLP e treinar modelos como BERT!', 'Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de treinar NLP e modelos como BERT!', 'Eu gosto de NLP e modelos de treinamento como BERT!'], ['Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de NLP e treinar modelos como BERT!', 'Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de treinar NLP e modelos como BERT!', 'Eu gosto de NLP e modelos de treinamento como BERT!'], ['Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de NLP e treinar modelos como BERT!', 'Eu gosto de NLP e treino modelos como BERT!', 'Eu gosto de treinar NLP e modelos como BERT!', 'Eu gosto de NLP e modelos de treinamento como BERT!']]


# Parafraseador

In [3]:
from nlpaug.augmenter.word import WordAugmenter
class PegasusPharaphraseAugmenterOtherLang(WordAugmenter):
    # https://arxiv.org/pdf/1511.06709.pdf
    """
    Augmenter that leverage two translation models for augmentation. For example, the source is English. This
    augmenter translate source to German and translating it back to English. For detail, you may visit
    https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28
    :param str from_model_name: Any model from https://huggingface.co/models?filter=translation&search=Helsinki-NLP. As
        long as from_model_name is pair with to_model_name. For example, from_model_name is English to Japanese,
        then to_model_name should be Japanese to English.
    :param str to_model_name: Any model from https://huggingface.co/models?filter=translation&search=Helsinki-NLP.
    :param str device: Default value is CPU. If value is CPU, it uses CPU for processing. If value is CUDA, it uses GPU
        for processing. Possible values include 'cuda' and 'cpu'. (May able to use other options)
    :param bool force_reload: Force reload the contextual word embeddings model to memory when initialize the class.
        Default value is False and suggesting to keep it as False if performance is the consideration.
    :param int batch_size: Batch size.
    :param int max_length: The max length of output text.
    :param str name: Name of this augmenter
    >>> import nlpaug.augmenter.word as naw
    >>> aug = naw.BackTranslationAug()
    """

    def __init__(self, src_lang='pt'  ,max_length=60,num_beams=10,temperature=1.5, num_return_sequences = 3,
        name='PegasusPharaphraseAugmenterOtherLang'):
        super().__init__(
            action='substitute')
        
        import torch
        from transformers import PegasusForConditionalGeneration, PegasusTokenizer
        model_name = 'tuner007/pegasus_paraphrase'
        torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

        tokenizer = PegasusTokenizer.from_pretrained(model_name)
        model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

        self.src_lang = 'pt'
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.num_beams = num_beams
        self.temperature = temperature
        self.torch_device = torch_device
        self.num_return_sequences = num_return_sequences

    def substitute(self, data, n = 10):
      from googletrans import Translator
      
      translator = Translator()
      result = translator.translate(data, src = self.src_lang, dest='en')
      
      
      batch = self.tokenizer(result.text,truncation=True,padding='longest',max_length=self.max_length, return_tensors="pt").to(self.torch_device)
      translated = self.model.generate(**batch,max_length=self.max_length,num_beams=self.num_beams, 
                                       num_return_sequences=self.num_return_sequences, temperature=self.temperature)
      tgt_text = self.tokenizer.batch_decode(translated, skip_special_tokens=True)

      rs_text = []

      for t in tgt_text:
        fim = translator.translate(t, src = 'en', dest = self.src_lang)
        rs_text.append(fim.text)
      
      return rs_text

In [4]:
aug = PegasusPharaphraseAugmenterOtherLang(src_lang='pt', num_return_sequences = 10)

In [5]:
augmented_texts = aug.augment(text)
print("Original:")
print(text)
print("Augmented Texts:")
print(augmented_texts)

Original:
Eu adoro NLP e treinar modelos como BERT!
Augmented Texts:
[['Gosto de treinar modelos como o BERT.', 'Eu gosto de modelos de treinamento como o BERT.', 'Sou um grande fã de modelos de treinamento como o BERT.', 'Gosto de treinar modelos como o BERT.', 'Adoro treinar modelos como o BERT.', 'O BERT é um modelo de treino que adoro.', 'Eu gosto de modelos de treinamento como o BERT.', 'BERT é um modelo de treino que adoro.', 'BERT é um modelo de treinamento que eu gosto.', 'BERT é um modelo de treinamento que eu gosto.']]
